In [ ]:
import csv
from pathlib import Path
from datetime import datetime
from collections import defaultdict

BANK_DIR = Path("dp700_bancos")  # Carpeta con los CSV generados

# -------------------------------
# Utilidades de carga y guardado
# -------------------------------

def cargar_preguntas_desde_bancos(bank_dir: Path):
    """
    Lee todos los CSV de la carpeta bank_dir y devuelve:
      - lista de preguntas
      - mapa archivo -> lista de filas originales para poder actualizar notas
    Cada CSV debe tener cabecera:
      section,id,question,options,correct,multi,notas
    """
    preguntas = []
    archivos_data = {}  # {Path: [dict_row]}

    if not bank_dir.exists():
        print(f"La carpeta {bank_dir} no existe. Ejecuta primero el generador de bancos.")
        return preguntas, archivos_data

    for csv_file in bank_dir.glob("*.csv"):
        with open(csv_file, mode="r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            rows = []
            for row in reader:
                rows.append(row)

                section = row["section"].strip()
                qid = int(row["id"])
                question = row["question"].strip()
                options = [opt.strip() for opt in row["options"].split(";")]
                correct_str = row["correct"].strip()
                if correct_str:
                    correct_indices = [int(x) - 1 for x in correct_str.split(";")]
                else:
                    correct_indices = []
                multi = row["multi"].strip().lower() == "true"
                notas = row.get("notas", "").strip()

                preguntas.append({
                    "section": section,
                    "id": qid,
                    "question": question,
                    "options": options,
                    "correct": sorted(correct_indices),
                    "multi": multi,
                    "notas": notas,
                    "source_file": csv_file,   # para saber qué archivo actualizar
                })
            archivos_data[csv_file] = rows

    return preguntas, archivos_data

def agrupar_por_seccion(preguntas):
    secciones = defaultdict(list)
    for q in preguntas:
        secciones[q["section"]].append(q)
    return secciones

def actualizar_notas_en_archivo(archivos_data, pregunta, nueva_nota):
    """
    Actualiza la columna 'notas' para la pregunta dada (por section + id)
    en el CSV de origen, agregando la nueva nota separada por '|'.
    """
    csv_path = pregunta["source_file"]
    if csv_path not in archivos_data:
        return

    rows = archivos_data[csv_path]
    target_section = pregunta["section"]
    target_id = str(pregunta["id"])

    for row in rows:
        if row["section"].strip() == target_section and row["id"].strip() == target_id:
            notas_actuales = row.get("notas", "").strip()
            if notas_actuales:
                row["notas"] = notas_actuales + "|" + nueva_nota
            else:
                row["notas"] = nueva_nota
            # actualizar también en la instancia en memoria
            pregunta["notas"] = row["notas"]
            break

    # reescribir el archivo completo con las notas actualizadas
    fieldnames = ["section", "id", "question", "options", "correct", "multi", "notas"]
    with open(csv_path, mode="w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            # asegurar que 'notas' existe
            if "notas" not in row:
                row["notas"] = ""
            writer.writerow({
                "section": row["section"],
                "id": row["id"],
                "question": row["question"],
                "options": row["options"],
                "correct": row["correct"],
                "multi": row["multi"],
                "notas": row.get("notas", ""),
            })

# -------------------------------
# Interfaz de consola
# -------------------------------

def clear_screen():
    print("\n" + "-" * 60 + "\n")

def mostrar_menu_secciones(secciones):
    clear_screen()
    print("=== DP-700 Quiz (bancos CSV) ===\n")
    keys = sorted(secciones.keys())
    for idx, sec in enumerate(keys, start=1):
        print(f"{idx}. {sec} ({len(secciones[sec])} preguntas)")
    print("R. Recargar bancos (leer de nuevo los CSV)")
    print("0. Salir\n")
    return keys

def elegir_opcion(keys):
    while True:
        choice = input("Elige opción: ").strip().upper()
        if choice == "R":
            return "R"
        if choice.isdigit():
            n = int(choice)
            if 0 <= n <= len(keys):
                return n
        print("Entrada inválida. Usa un número o 'R'.")

def seleccionar_preguntas(preguntas_seccion, minimo=5, maximo=15):
    total = len(preguntas_seccion)
    if total < minimo:
        print(f"\nLa sección tiene solo {total} preguntas (mínimo recomendado: {minimo}).")
    return preguntas_seccion[:min(total, maximo)]

def pedir_respuesta_o_nota(q, archivos_data):
    """
    Muestra la pregunta y permite:
      - introducir respuesta (números / enter)
      - pulsar 'N' para agregar nota antes de responder
    Devuelve la lista de índices elegidos para la respuesta.
    """
    while True:
        print(f"\nPregunta {q['id']}: {q['question']}\n")
        for idx, opt in enumerate(q["options"], start=1):
            print(f"  {idx}) {opt}")

        if q["notas"]:
            print(f"\nNotas actuales: {q['notas']}")

        if q["multi"]:
            print("\n(Puede haber varias correctas. Separa con comas, ej: 1,3,4)")
        else:
            print("\n(Selecciona una sola opción)")

        print("Pulsa 'N' para agregar nota a esta pregunta.")

        ans = input("Tu respuesta (o N para nota): ").strip()

        # Manejo de notas
        if ans.upper() == "N":
            nota = input("Escribe tu nota y pulsa Enter: ").strip()
            if nota:
                actualizar_notas_en_archivo(archivos_data, q, nota)
                print("Nota guardada.\n")
            # luego vuelve a mostrar la pregunta para responderla
            continue

        if not ans:
            print("La respuesta no puede estar vacía.")
            continue

        parts = [p.strip() for p in ans.replace(";", ",").split(",")]
        if all(p.isdigit() for p in parts):
            idxs = [int(p) - 1 for p in parts]
            if all(0 <= i < len(q["options"]) for i in idxs):
                return sorted(list(set(idxs)))

        print("Entrada inválida. Usa números de opciones (ej: 2 o 1,3) o 'N' para nota.")

def evaluar(q, user_idx):
    return user_idx == q["correct"]

def puntuacion(num_ok, total):
    if total == 0:
        return 0.0
    return round(100.0 * num_ok / total, 2)

# -------------------------------
# Persistencia de resultados
# -------------------------------

def nombre_csv_resultados(section_name):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    safe = section_name.replace(" ", "_").replace("/", "_")
    return f"dp700_resultados_{safe}_{ts}.csv"

def guardar_resultados_csv(section_name, respuestas, score):
    filename = nombre_csv_resultados(section_name)
    campos = [
        "timestamp",
        "section",
        "question_id",
        "is_correct",
        "user_answer_indices",
        "correct_indices",
        "score_session"
    ]
    ts_now = datetime.now().isoformat(timespec="seconds")

    with open(filename, mode="w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writeheader()
        for r in respuestas:
            writer.writerow({
                "timestamp": ts_now,
                "section": section_name,
                "question_id": r["question_id"],
                "is_correct": r["is_correct"],
                "user_answer_indices": ";".join(str(i) for i in r["user_answer_indices"]),
                "correct_indices": ";".join(str(i) for i in r["correct_indices"]),
                "score_session": score
            })

    print(f"\nCSV de resultados generado: {filename}")

# -------------------------------
# Bucle principal
# -------------------------------

def run():
    preguntas, archivos_data = cargar_preguntas_desde_bancos(BANK_DIR)
    if not preguntas:
        print("No se pudieron cargar preguntas. Revisa la carpeta dp700_bancos.")
        return

    secciones = agrupar_por_seccion(preguntas)

    while True:
        keys = mostrar_menu_secciones(secciones)
        choice = elegir_opcion(keys)

        if choice == "R":
            preguntas, archivos_data = cargar_preguntas_desde_bancos(BANK_DIR)
            secciones = agrupar_por_seccion(preguntas)
            print("\nBancos recargados desde los CSV.")
            input("Enter para continuar...")
            continue

        if choice == 0:
            print("Saliendo. ¡Buen estudio para el DP-700!")
            break

        section_name = keys[choice - 1]
        clear_screen()
        print(f"=== Sección: {section_name} ===")

        preguntas_seccion = seleccionar_preguntas(secciones[section_name], minimo=5, maximo=15)
        if not preguntas_seccion:
            print("No hay preguntas para esta sección.")
            input("Enter para volver al menú...")
            continue

        respuestas_log = []
        correctas = 0

        for q in preguntas_seccion:
            user_idx = pedir_respuesta_o_nota(q, archivos_data)
            ok = evaluar(q, user_idx)
            if ok:
                print("✔ Correcto\n")
                correctas += 1
            else:
                print("✘ Incorrecto\n")
                add_note = input("¿Quieres agregar una nota? (S/N): ").strip().upper()
                if add_note == "S":
                    nota = input("Escribe tu nota: ").strip()
                    if nota:
                        actualizar_notas_en_archivo(archivos_data, q, nota)
                        print("Nota guardada.")

            respuestas_log.append({
                "question_id": q["id"],
                "is_correct": ok,
                "user_answer_indices": user_idx,
                "correct_indices": q["correct"],
            })

        score = puntuacion(correctas, len(preguntas_seccion))
        print(f"\nPuntuación sesión: {score}/100")
        guardar_resultados_csv(section_name, respuestas_log, score)

        input("\nEnter para volver al menú...")

if __name__ == "__main__":
    run()


------------------------------------------------------------

=== DP-700 Quiz (bancos CSV) ===

1. Implement Real-Time Intelligence with Microsoft Fabric (5 preguntas)
2. Implement a Data Warehouse with Microsoft Fabric (5 preguntas)
3. Implement a Lakehouse with Microsoft Fabric (5 preguntas)
4. Ingest Data with Microsoft Fabric (5 preguntas)
5. Manage a Microsoft Fabric environment (5 preguntas)
R. Recargar bancos (leer de nuevo los CSV)
0. Salir


------------------------------------------------------------

=== Sección: Implement Real-Time Intelligence with Microsoft Fabric ===

Pregunta 1: What is an Eventstream used for in Fabric?

  1) Routing and transforming real-time events
  2) Batch ETL only
  3) Email processing
  4) User authentication

(Selecciona una sola opción)
Pulsa 'N' para agregar nota a esta pregunta.
✔ Correcto


Pregunta 2: Which components are typical in RTI solutions?

  1) Eventstream
  2) KQL database
  3) Real-time dashboard
  4) VM scale sets

(Puede habe